In [1]:
import sys
sys.path.insert(0, '../../Utilities/')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
# from plotting import newfig, savefig あとで追加
from mpl_toolkits.mplot3d import Axes3D
import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
np.random.seed(1234)
tf.random.set_seed(1234)

In [7]:
class PhysicsInformedNN:
    #クラスの初期化
    def __init__(self, X_u, u, X_f, layers, lb, ub, nu):
        #Todo：それぞれの変数について説明を追加する。
        self.lb = lb
        self.ub = ub

        #データのセット
        #[:,0:1]は全ての行と0列目のデータを取得
        #第0列（最初の列）から第1列の直前までを選択します。実質的には第0列だけを選択
        self.x_u = X_u[:,0:1]
        self.t_u = X_u[:,1:2]

        self.x_f = X_f[:,0:1]
        self.t_f = X_f[:,1:2]

        self.u = u

        self.layers = layers
        self.nu = nu

        #NNの重みとバイアスの初期化
        self.weights, self.biases = self.initialize_NN(layers)

        #セッションの作成
        #ConfigProto オプションで、ハードウェアに適応するように設定しています。
        #allow_soft_placement=True：指定されたデバイスが使用できない場合に、別のデバイスに割り当てます。
        #log_device_placement=True：操作がどのデバイスに配置されるかをログに出力します。
        # self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
        #                                              log_device_placement=True))
        
        #placeholderの作成
        #データをセッションにフィードするためのプレースホルダーを定義
        #x_u_tf, t_u_tf, u_tf は観測データの入力と出力。
        # self.x_u_tf = tf.placeholder(tf.float32, shape=[None, self.x_u.shape[1]])
        # self.t_u_tf = tf.placeholder(tf.float32, shape=[None, self.t_u.shape[1]])        
        # self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
        # #微分方程式のための入力。
        # self.x_f_tf = tf.placeholder(tf.float32, shape=[None, self.x_f.shape[1]])
        # self.t_f_tf = tf.placeholder(tf.float32, shape=[None, self.t_f.shape[1]]) 

        #予測値の計算
        #ネットワーク net_u を使用して、観測データに対する予測値 u_pred を計算。
        self.u_pred = self.net_u(self.x_u_tf, self.t_u_tf)
        #ネットワーク net_f を使用して、微分方程式の残差 f_pred を計算。
        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)

        # 損失関数の定義
        #観測データと予測値の差の平均二乗誤差、および微分方程式の残差の平均二乗誤差を損失関数として定義。
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_mean(tf.square(self.f_pred))
               
        #ScipyOptimizerInterface を使用して、L-BFGS-B というスカラーベースの最適化方法で損失関数を最小化します。
        # 最大反復回数や関数評価回数などのオプションを設定
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})
        
        #変数の初期化
        #すべての TensorFlow 変数を初期化します。
        # セッション内で初期化を実行します。
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers)
        #重みとバイアスの初期化ループ
        #各層間の重みとバイアスを初期化するためのループ。range(0, num_layers - 1)は、入力層から最後の隠れ層までを対象にしている（出力層は含まない）。
        for l in range(0, num_layers - 1):
            #self.xavier_initは、Xavier初期化を用いて重み行列Wを生成するメソッド
            #sizeパラメータは、行列のサイズを指定。[layers[l], layers[l+1]]は、l層からl+1層への重み行列の形状を示す。
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            # tf.zerosは、layers[l+1]ユニット分のバイアスベクトルをゼロで初期化します。
            # tf.Variableは、このゼロベクトルをテンソルフローの変数として定義します。
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases

     # ニューラルネットワークの重みの初期化を行う関数    
    def xavier_init(self, size):
        #重み行列の入力側と出力側の次元数
        in_dim = size[0]
        out_dim = size[1] 
        #重みをランダムに初期化する際の標準偏差       
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    # 多層パーセプトロンNNの順伝播
    #X:入力データ, weights:NNの各層の重みを含むリスト, biases:各層のバイアスを含むリスト
    def neural_net(self, X, weights, biases):
        #層数の計算
        num_layers = len(weights) + 1
        #入力データの正規化
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        #隠れ層の計算
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        #出力層の計算
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y

    #xとtを結合してニューラルネットワークに通し、その出力を返すメソッド。1つ上のneural_netメソッドを呼び出す
    def net_u(self, x, t):
        u = self.neural_net(tf.concat([x,t],1), self.weights, self.biases)
        return u
    
    # PINNでよく用いられる形式。偏微分方程式の残差を計算するメソッド
    def net_f(self, x,t):
        #ニューラルネットワークにxとtを入力してuを計算
        u = self.net_u(x,t)
        #偏微分方程式の残差を計算
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        #PDEの残差の計算
        f = u_t + u*u_x - self.nu*u_xx
        
        return f
    
    def callback(self, loss):
        print('Loss:', loss)

     #ニューラルネットワークのトレーニングを行うメソッド
    def train(self):
        
        tf_dict = {self.x_u_tf: self.x_u, self.t_u_tf: self.t_u, self.u_tf: self.u,
                   self.x_f_tf: self.x_f, self.t_f_tf: self.t_f}
                                                                                                                          
        self.optimizer.minimize(self.sess, 
                                feed_dict = tf_dict,         
                                fetches = [self.loss], 
                                loss_callback = self.callback)        
                                    
    #予測を行うメソッド
    def predict(self, X_star):
                
        u_star = self.sess.run(self.u_pred, {self.x_u_tf: X_star[:,0:1], self.t_u_tf: X_star[:,1:2]})  
        f_star = self.sess.run(self.f_pred, {self.x_f_tf: X_star[:,0:1], self.t_f_tf: X_star[:,1:2]})
               
        return u_star, f_star
                


In [8]:
if __name__ == "__main__": 
     
    nu = 0.01/np.pi
    noise = 0.0        

    N_u = 100
    N_f = 10000
    layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]
    
    data = scipy.io.loadmat('../Data/burgers_shock.mat')
    
    t = data['t'].flatten()[:,None]
    x = data['x'].flatten()[:,None]
    Exact = np.real(data['usol']).T
    
    X, T = np.meshgrid(x,t)
    
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.flatten()[:,None]              

    # Doman bounds
    lb = X_star.min(0)
    ub = X_star.max(0)    
        
    xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
    uu1 = Exact[0:1,:].T
    xx2 = np.hstack((X[:,0:1], T[:,0:1]))
    uu2 = Exact[:,0:1]
    xx3 = np.hstack((X[:,-1:], T[:,-1:]))
    uu3 = Exact[:,-1:]
    
    X_u_train = np.vstack([xx1, xx2, xx3])
    X_f_train = lb + (ub-lb)*lhs(2, N_f)
    X_f_train = np.vstack((X_f_train, X_u_train))
    u_train = np.vstack([uu1, uu2, uu3])
    
    idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)
    X_u_train = X_u_train[idx, :]
    u_train = u_train[idx,:]
        
    model = PhysicsInformedNN(X_u_train, u_train, X_f_train, layers, lb, ub, nu)
    
    start_time = time.time()                
    model.train()
    elapsed = time.time() - start_time                
    print('Training time: %.4f' % (elapsed))
    
    u_pred, f_pred = model.predict(X_star)
            
    error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
    print('Error u: %e' % (error_u))                     

    
    U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
    Error = np.abs(Exact - U_pred)

AttributeError: 'PhysicsInformedNN' object has no attribute 'x_u_tf'